In [1]:
import cv2
import numpy as np
import os

In [2]:
# Paths
input_video_path = './updated_board_output.mp4'
output_video_path = 'smartboard_output/smartboard_output.mp4'

def compute_output_size(pts):
    tl, tr, br, bl = pts
    width_top = np.linalg.norm(tr - tl)
    width_bottom = np.linalg.norm(br - bl)
    width = int(max(width_top, width_bottom))

    height_left = np.linalg.norm(bl - tl)
    height_right = np.linalg.norm(br - tr)
    height = int(max(height_left, height_right))

    return (width, height)

def save_frame_with_points_and_polygon(frame, selected_points, output_path):
    """
    Saves a copy of the frame with:
    - Green circles on the selected points
    - A red polygon connecting the four points (in order)
    """
    frame_copy = frame.copy()

    # Draw green dots at each selected point
    for pt in selected_points:
        cv2.circle(frame_copy, pt, 8, (0, 255, 0), -1)

    # Draw red polygon connecting the selected points
    pts_array = np.array(selected_points, dtype=np.int32).reshape((-1, 1, 2))
    cv2.polylines(frame_copy, [pts_array], isClosed=True, color=(0, 0, 255), thickness=2)

    # Save the annotated frame
    cv2.imwrite(output_path, frame_copy)
    print(f"Saved: {output_path}")



# Open video
cap = cv2.VideoCapture(input_video_path)
video_fps = cap.get(cv2.CAP_PROP_FPS)
assert cap.isOpened(), "Failed to open input video."

# Read first frame
ret, first_frame = cap.read()
if not ret:
    raise ValueError("Failed to read from video.")

# Global list to store points
selected_points = []

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_points.append((x, y))
        cv2.circle(first_frame, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow('Select 4 points (Top-Left, Top-Right, Bottom-Right, Bottom-Left)', first_frame)

# Show frame and let user pick 4 points
window_name = 'Select 4 points (Top-Left, Top-Right, Bottom-Right, Bottom-Left)'
cv2.imshow(window_name, first_frame)
cv2.setMouseCallback(window_name, click_event)


print("Click on the 4 corners of the whiteboard (Top-Left, Top-Right, Bottom-Right, Bottom-Left)")

while True:
    if len(selected_points) == 4:
        break
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key to quit early
        break

cv2.destroyAllWindows()
# Draw circles and bounding box on a copy of the first frame
frame_with_points = first_frame.copy()

# Draw green dots for each point
for pt in selected_points:
    cv2.circle(frame_with_points, pt, 8, (0, 255, 0), -1)

# Compute bounding box
pts_array = np.array(selected_points, dtype=np.int32)
x, y, w, h = cv2.boundingRect(pts_array)

# Draw bounding box in red
cv2.rectangle(frame_with_points, (x, y), (x + w, y + h), (0, 0, 255), 2)

# Save image for the presentation
save_frame_with_points_and_polygon(first_frame, selected_points, "smartboard_output/selected_frame_with_dots_and_polygon.jpg")
print("Saved: smartboard_output/selected_frame_with_dots_and_box.jpg")

if len(selected_points) != 4:
    raise ValueError("Exactly 4 points must be selected!")

# Prepare source and destination points
src_pts = np.array(selected_points, dtype=np.float32)
output_size = compute_output_size(src_pts)

dst_pts = np.array([
    [0, 0],
    [output_size[0]-1, 0],
    [output_size[0]-1, output_size[1]-1],
    [0, output_size[1]-1]
], dtype=np.float32)

M = cv2.getPerspectiveTransform(src_pts, dst_pts)

# Prepare output video writer
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Rewind to start
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
smartboard_writer = cv2.VideoWriter(output_video_path, fourcc, video_fps, output_size)

# Process all frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    warped_frame = cv2.warpPerspective(frame, M, output_size)
    smartboard_writer.write(warped_frame)

# Release everything
cap.release()
smartboard_writer.release()

print(f"Smartboard video saved at: {output_video_path}")


Click on the 4 corners of the whiteboard (Top-Left, Top-Right, Bottom-Right, Bottom-Left)
Saved: smartboard_output/selected_frame_with_dots_and_polygon.jpg
Saved: smartboard_output/selected_frame_with_dots_and_box.jpg
Smartboard video saved at: smartboard_output/smartboard_output.mp4
